In [1]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
The token `Coli_project` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate w

In [2]:
!pip install -qq adapters
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 102.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 19.4 MB/s eta 0:00:00


In [3]:
import pandas as pd
from adapters import AutoAdapterModel, AdapterTrainer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.utils import shuffle
from transformers import EvalPrediction
from transformers import EarlyStoppingCallback, IntervalStrategy, TrainingArguments
import numpy as np
import torch
import transformers
from transformers import AutoTokenizer
from transformers import AutoConfig
from typing import Dict, Any
from optuna import Trial
from typing import Dict, Union
import json
from google.colab import files

In [4]:
# This function saves the performance of the models
def write_json(file_name, data):
    with open(file_name, mode="w") as f:
        json_str = json.dumps(data)
        f.write(json_str)

def load_json(file_name):
    data = {}
    with open(file_name, mode="r") as f:
        data = json.load(f)
    return data

In [5]:
def create_sentiment_analysis_model():
    config = AutoConfig.from_pretrained("xlm-roberta-base", num_labels=3)
    model = AutoAdapterModel.from_pretrained("xlm-roberta-base", config=config)

    # Here I am adding adapter for sentiment analysis
    model.add_adapter("Telugu_code_mix_sentimentanalysis", config="pfeiffer")
    model.add_classification_head("Telugu_code_mix_sentimentanalysis", num_labels=3)

    # Here I am activating the dapter
    model.train_adapter("Telugu_code_mix_sentimentanalysis")
    return model

In [6]:
#code from huggingface
#https://huggingface.co/docs/setfit/en/how_to/hyperparameter_optimization
def hp_space(trial: Trial) -> Dict[str, Union[float, int, str]]:
    return {
        "body_learning_rate": trial.suggest_float("body_learning_rate", 1e-6, 1e-3, log=True),
        "num_epochs": trial.suggest_int("num_epochs", 1, 10),
        "batch_size": trial.suggest_categorical("batch_size", [16, 32, 64]),
        "seed": trial.suggest_int("seed", 1, 40),
        "max_iter": trial.suggest_int("max_iter", 50, 300),
        "solver": trial.suggest_categorical("solver", ["newton-cg", "lbfgs", "liblinear"]),
    }

In [7]:
df = pd.read_excel("/content/train_set(2).xlsx")

In [8]:
df.head()

,Label,Text,English_Translation,Telugu_Translation
0,NTL,We need Mr chari s review on master,We need Mr Charis review on Master,మాస్టర్పై చారి గారి సమీక్ష కావాలి
1,NEG,worst government YSRCP chala chethha ga parip...,Worst government YSRCP is managing very badly,వర్స్ట్ ప్రభుత్వం YSRCP చాలా చెత్తగా పరిపాలన ...
2,NEG,bayya nuvvu emina cheppu kani bagoledu ani che...,You tell me what to say but its difficult to s...,బయ్య నువ్వు ఏమినా చెప్పు కనీ బగోలెడు అని చెప్ప...
3,POS,Dube gadini vadilesi manchhi Pani chesaru RCBT...,The Dube river is overflowing and RCB has done...,డ్యూబే గడిని వడిలేసి మంచి పని చేశారు ఆర్సీబీ ట...
4,POS,I came to watch thyview s review crying after ...,I came to watch Thyviews review crying after w...,నేను దిల్ బేచారా చూసిన తర్వాత థైవ్యూ సమీక్ష చూ...


In [9]:
#Here , I am loading the data
df['Label'] = LabelEncoder().fit_transform(df['Label'])
df = shuffle(df)

#Here I split the data to train and test(Validation split)
training_texts, validation_texts, training_labels, validation_labels = train_test_split(df['Text'].tolist(), df['Label'].tolist(), test_size=0.2)

#Here I tokenized the data
text_tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

# Here, I am making sure the text is in  string format
training_texts = [str(text) for text in training_texts]
validation_texts = [str(text) for text in validation_texts]

def tokenization(texts, labels):
    tokenized = text_tokenizer(texts, max_length=80, truncation=True, padding="max_length")
    tokenized['labels'] = labels
    return tokenized

train_tokenized = tokenization(training_texts, training_labels)
val_tokenized = tokenization(validation_texts, validation_labels)

# Here i am craeting the pytorch dataloader
class my_coli_Dataset(torch.utils.data.Dataset):
    def __init__(self, tokenized):
        self.tokenized = tokenized

    def __getitem__(self, index):
        item = {key: torch.tensor(value[index]) for key, value in self.tokenized.items()}
        return item

    def __len__(self):
        return len(self.tokenized['input_ids'])

train_dataset = my_coli_Dataset(train_tokenized)
val_dataset = my_coli_Dataset(val_tokenized)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [10]:
# Here I am loading test dataset
test_dataframe = pd.read_excel("/content/test_set(1).xlsx")

# Here I am extracting the samples and labels
test_samples = list(test_dataframe['Text'])
test_dataframe['Label'] = LabelEncoder().fit_transform(test_dataframe.Label.values)
encoded_labels = test_dataframe['Label']

# Here I am processing the text data
tokenized_test_data = tokenization(test_samples, encoded_labels)

# Here I am craeting the pytorch test custom dataset
test_custom_dataset = my_coli_Dataset(tokenized_test_data)

In [11]:
import os
os.environ["WANDB_DISABLED"] = "true"

##First Hyperparameter tuning

In [12]:
#1st Hyperparameter tuning
training_arguments = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
)

trainer = AdapterTrainer(
    model_init=create_sentiment_analysis_model,
    args=training_arguments,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=text_tokenizer,
)

best_run = trainer.hyperparameter_search(direction="minimize", hp_space=hp_space, n_trials=10)
print(best_run)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaAdapterModel were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
[I 2025-03-31 12:34:32,256] A new study created in memory with name: no-name-e1c9a23d-df11-41ed-bafd-7b7ebb400e99
Trying to set body_learning_rate in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set num_epochs in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set batch_size in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set max_iter in the hyperparameter search but there is no 

Epoch,Training Loss,Validation Loss
1,0.869000,0.771329
2,0.708300,0.595389
3,0.621200,0.576895


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
[I 2025-03-31 12:38:36,984] Trial 0 finished with value: 0.5768945813179016 and parameters: {'body_learning_rate': 8.969678113361717e-05, 'num_epochs': 4, 'batch_size': 32, 'seed': 25, 'max_iter': 269, 'solver': 'newton-cg'}. Best is trial 0 with value: 0.5768945813179016.
Trying to set body_learning_rate in th

...

Epoch,Training Loss,Validation Loss
1,0.879300,0.749909
2,0.741200,0.624469


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
[I 2025-03-31 13:04:40,821] Trial 8 pruned. 
Trying to set body_learning_rate in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set num_epochs in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set batch_size in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set max_iter in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set solver in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Some weights of XLMRobertaAdapterModel were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream

Epoch,Training Loss,Validation Loss
1,0.864100,0.754219


[I 2025-03-31 13:06:04,423] Trial 9 pruned. 


BestRun(run_id='7', objective=0.5470589995384216, hyperparameters={'body_learning_rate': 6.931001379744486e-05, 'num_epochs': 8, 'batch_size': 16, 'seed': 2, 'max_iter': 260, 'solver': 'lbfgs'}, run_summary=None)


In [13]:
config = AutoConfig.from_pretrained("xlm-roberta-base", num_labels=3)
model = AutoAdapterModel.from_pretrained("xlm-roberta-base", config=config)


#Here I am adding adapter
model.add_adapter("Telugu_code_mix_sentimentanalysis", config="pfeiffer")

model.add_classification_head("Telugu_code_mix_sentimentanalysis", num_labels=3)

#Here I am activating adapter
model.train_adapter("Telugu_code_mix_sentimentanalysis")

Some weights of XLMRobertaAdapterModel were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
#Here, I fine tune Telugu codemix data with the best hyperparameters
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {'Accuracy': acc,
            'F1': f1,
            'Precision': precision,
            'Recall': recall
           }

training_arguments = TrainingArguments(
    learning_rate=best_run.hyperparameters['body_learning_rate'],
    num_train_epochs=best_run.hyperparameters['num_epochs'],
    per_device_train_batch_size=best_run.hyperparameters['batch_size'],
    per_device_eval_batch_size=best_run.hyperparameters['batch_size'],
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True,
    remove_unused_columns=False,
)

trainer = AdapterTrainer(
    model=model.to('cuda'),
    args=training_arguments,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss
200,1.000500
400,0.908800
600,0.875900
800,0.827200
1000,0.788500
1200,0.736300
1400,0.671800
1600,0.671600
1800,0.617800
2000,0.592900


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

TrainOutput(global_step=6752, training_loss=0.5799690185564954, metrics={'train_runtime': 336.8215, 'train_samples_per_second': 320.502, 'train_steps_per_second': 20.046, 'total_flos': 4558656593687040.0, 'train_loss': 0.5799690185564954, 'epoch': 8.0})

In [15]:
trainer.evaluate()

{'eval_loss': 0.4784475266933441,
 'eval_Accuracy': 0.7972732661529343,
 'eval_F1': 0.7725699313656736,
 'eval_Precision': 0.7716180096273592,
 'eval_Recall': 0.7737451793914021,
 'eval_runtime': 4.4762,
 'eval_samples_per_second': 753.759,
 'eval_steps_per_second': 47.138,
 'epoch': 8.0}

In [17]:
test_results = trainer.evaluate(test_custom_dataset)

In [18]:
print(test_results)

{'eval_loss': 0.5692409873008728, 'eval_Accuracy': 0.7663333333333333, 'eval_F1': 0.7640583350228094, 'eval_Precision': 0.7683793523227337, 'eval_Recall': 0.7663333333333333, 'eval_runtime': 4.1801, 'eval_samples_per_second': 717.683, 'eval_steps_per_second': 44.975, 'epoch': 8.0}


In [19]:
write_json('Telugu_code_mix_sentimentanalysis_hyper_parameter_1.json',best_run.hyperparameters)
files.download('Telugu_code_mix_sentimentanalysis_hyper_parameter_1.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [20]:
write_json('Telugu_code_mix_sentimentanalysis_result1.json',test_results)
files.download('Telugu_code_mix_sentimentanalysis_result1.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [21]:
model.save_adapter("./final_adapter", "Telugu_code_mix_sentimentanalysis")
!ls -lh final_adapter

total 5.8M
-rw-r--r-- 1 root root 1.1K Mar 31 13:18 adapter_config.json
-rw-r--r-- 1 root root  491 Mar 31 13:18 head_config.json
-rw-r--r-- 1 root root 3.5M Mar 31 13:18 pytorch_adapter.bin
-rw-r--r-- 1 root root 2.3M Mar 31 13:18 pytorch_model_head.bin


In [22]:
model.push_adapter_to_hub(
    repo_id="my-Telugu-codemix-xlmr-adapter1",
    adapter_name="Telugu_code_mix_sentimentanalysis",
)

pytorch_adapter.bin:   0%|          | 0.00/3.60M [00:00<?, ?B/s]

pytorch_model_head.bin:   0%|          | 0.00/2.37M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/Blue7Bird/my-Telugu-codemix-xlmr-adapter1/commit/80f4c76b9c8e696a9f6cf402d6d9d6f56fc253e6', commit_message='Upload model', commit_description='', oid='80f4c76b9c8e696a9f6cf402d6d9d6f56fc253e6', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Blue7Bird/my-Telugu-codemix-xlmr-adapter1', endpoint='https://huggingface.co', repo_type='model', repo_id='Blue7Bird/my-Telugu-codemix-xlmr-adapter1'), pr_revision=None, pr_num=None)

###second hyperparameter tuning

In [23]:
#2nd Hyperparameter tuning
training_arguments = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
)

trainer = AdapterTrainer(
    model_init=create_sentiment_analysis_model,
    args=training_arguments,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=text_tokenizer,
)

best_run = trainer.hyperparameter_search(direction="minimize", hp_space=hp_space, n_trials=10)
print(best_run)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Some weights of XLMRobertaAdapterModel were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
[I 2025-03-31 13:21:42,185] A new study created in memory with name: no-name-8ea85dde-7764-48ff-98df-24faa

Epoch,Training Loss,Validation Loss
1,0.865400,0.768604
2,0.713200,0.578086
3,0.614800,0.559069


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
[I 2025-03-31 13:25:51,579] Trial 0 finished with value: 0.5590687394142151 and parameters: {'body_learning_rate': 9.391380452265998e-05, 'num_epochs': 6, 'batch_size': 32, 'seed': 18, 'max_iter': 228, 'solver': 'lbfgs'}. Best is trial 0 with value: 0.5590687394142151.
Trying to set body_learning_rate in the hy

Epoch,Training Loss,Validation Loss
1,0.858100,0.737020
2,0.725400,0.617044
3,0.664800,0.597972


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
[I 2025-03-31 13:30:03,740] Trial 1 finished with value: 0.5979722142219543 and parameters: {'body_learning_rate': 0.0007182303316033247, 'num_epochs': 3, 'batch_size': 16, 'seed': 38, 'max_iter': 142, 'solver': 'lbfgs'}. Best is trial 0 with value: 0.5590687394142151.
Trying to set body_learning_rate in the hy

...

Epoch,Training Loss,Validation Loss
1,0.854900,0.736825
2,0.705500,0.604766


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
[I 2025-03-31 13:53:36,455] Trial 8 pruned. 
Trying to set body_learning_rate in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set num_epochs in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set batch_size in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set max_iter in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set solver in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Some weights of XLMRobertaAdapterModel were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream

Epoch,Training Loss,Validation Loss
1,0.862600,0.752302


[I 2025-03-31 13:54:59,961] Trial 9 pruned. 


BestRun(run_id='0', objective=0.5590687394142151, hyperparameters={'body_learning_rate': 9.391380452265998e-05, 'num_epochs': 6, 'batch_size': 32, 'seed': 18, 'max_iter': 228, 'solver': 'lbfgs'}, run_summary=None)


In [24]:
config = AutoConfig.from_pretrained("xlm-roberta-base", num_labels=3)
model = AutoAdapterModel.from_pretrained("xlm-roberta-base", config=config)


#Here I am adding adapter
model.add_adapter("Telugu_code_mix_sentimentanalysis", config="pfeiffer")

model.add_classification_head("Telugu_code_mix_sentimentanalysis", num_labels=3)

#Here I am activating adapter
model.train_adapter("Telugu_code_mix_sentimentanalysis")

Some weights of XLMRobertaAdapterModel were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
#2nd training
training_arguments = TrainingArguments(
    learning_rate=best_run.hyperparameters['body_learning_rate'],
    num_train_epochs=best_run.hyperparameters['num_epochs'],
    per_device_train_batch_size=best_run.hyperparameters['batch_size'],
    per_device_eval_batch_size=best_run.hyperparameters['batch_size'],
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True,
    remove_unused_columns=False,
)

trainer = AdapterTrainer(
    model=model.to('cuda'),
    args=training_arguments,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss
200,0.976500
400,0.888300
600,0.813800
800,0.731200
1000,0.648100
1200,0.609300
1400,0.575100
1600,0.551000
1800,0.534800
2000,0.521100


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


TrainOutput(global_step=2532, training_loss=0.6491259534212085, metrics={'train_runtime': 199.6315, 'train_samples_per_second': 405.567, 'train_steps_per_second': 12.683, 'total_flos': 3418992445265280.0, 'train_loss': 0.6491259534212085, 'epoch': 6.0})

In [37]:
trainer.evaluate()

{'eval_loss': 0.46134185791015625,
 'eval_Accuracy': 0.8180201541197392,
 'eval_F1': 0.7927639663042679,
 'eval_Precision': 0.7912997594598257,
 'eval_Recall': 0.7946991585087136,
 'eval_runtime': 3.8072,
 'eval_samples_per_second': 886.224,
 'eval_steps_per_second': 27.842,
 'epoch': 7.0}

In [38]:
test_results = trainer.evaluate(test_custom_dataset)

In [39]:
print(test_results)

{'eval_loss': 0.5599409341812134, 'eval_Accuracy': 0.785, 'eval_F1': 0.7830406168387932, 'eval_Precision': 0.7853112323373431, 'eval_Recall': 0.785, 'eval_runtime': 3.4501, 'eval_samples_per_second': 869.53, 'eval_steps_per_second': 27.245, 'epoch': 7.0}


In [29]:
write_json('Telugu_code_mix_sentimentanalysis_hyper_parameter_2.json',best_run.hyperparameters)
files.download('Telugu_code_mix_sentimentanalysis_hyper_parameter_2.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [30]:
write_json('Telugu_code_mix_sentimentanalysis_result2.json',test_results)
files.download('Telugu_code_mix_sentimentanalysis_result2.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [32]:
model.save_adapter("./final_adapter", "Telugu_code_mix_sentimentanalysis")
!ls -lh final_adapter

total 5.8M
-rw-r--r-- 1 root root 1.1K Mar 31 14:07 adapter_config.json
-rw-r--r-- 1 root root  491 Mar 31 14:07 head_config.json
-rw-r--r-- 1 root root 3.5M Mar 31 14:07 pytorch_adapter.bin
-rw-r--r-- 1 root root 2.3M Mar 31 14:07 pytorch_model_head.bin


In [33]:
model.push_adapter_to_hub(
    repo_id="my-Telugu-codemix-xlmr-adapter2",
    adapter_name="Telugu_code_mix_sentimentanalysis",
)

pytorch_model_head.bin:   0%|          | 0.00/2.37M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_adapter.bin:   0%|          | 0.00/3.60M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Blue7Bird/my-Telugu-codemix-xlmr-adapter2/commit/66726217d8492eb002b5feeba077d29d76a47126', commit_message='Upload model', commit_description='', oid='66726217d8492eb002b5feeba077d29d76a47126', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Blue7Bird/my-Telugu-codemix-xlmr-adapter2', endpoint='https://huggingface.co', repo_type='model', repo_id='Blue7Bird/my-Telugu-codemix-xlmr-adapter2'), pr_revision=None, pr_num=None)

#Third hyperparameter tuning

In [34]:
#3rd Hyperparameter tuning
training_arguments = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
)

trainer = AdapterTrainer(
    model_init=create_sentiment_analysis_model,
    args=training_arguments,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=text_tokenizer,
)

best_run = trainer.hyperparameter_search(direction="minimize", hp_space=hp_space, n_trials=10)
print(best_run)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Some weights of XLMRobertaAdapterModel were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
[I 2025-03-31 14:09:32,324] A new study created in memory with name: no-name-8b57604e-e458-4993-87a9-df224

Epoch,Training Loss,Validation Loss
1,0.870500,0.738122
2,0.722000,0.630638
3,0.650100,0.574009


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
[I 2025-03-31 14:13:43,193] Trial 0 finished with value: 0.574009358882904 and parameters: {'body_learning_rate': 0.00017408932016542091, 'num_epochs': 5, 'batch_size': 32, 'seed': 30, 'max_iter': 153, 'solver': 'newton-cg'}. Best is trial 0 with value: 0.574009358882904.
Trying to set body_learning_rate in the

Epoch,Training Loss,Validation Loss
1,0.851600,0.725481
2,0.695000,0.594476
3,0.641600,0.568947


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
[I 2025-03-31 14:17:54,569] Trial 1 finished with value: 0.5689471960067749 and parameters: {'body_learning_rate': 3.5187705761490084e-06, 'num_epochs': 6, 'batch_size': 64, 'seed': 22, 'max_iter': 66, 'solver': 'newton-cg'}. Best is trial 1 with value: 0.5689471960067749.
Trying to set body_learning_rate in th

...

Epoch,Training Loss,Validation Loss
1,0.854900,0.736825


[I 2025-03-31 14:36:09,534] Trial 8 pruned. 
Trying to set body_learning_rate in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set num_epochs in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set batch_size in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set max_iter in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set solver in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Some weights of XLMRobertaAdapterModel were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,0.861800,0.758647


[I 2025-03-31 14:37:34,313] Trial 9 pruned. 


BestRun(run_id='3', objective=0.5590144991874695, hyperparameters={'body_learning_rate': 0.0004820940338739241, 'num_epochs': 7, 'batch_size': 32, 'seed': 24, 'max_iter': 122, 'solver': 'liblinear'}, run_summary=None)


In [35]:
config = AutoConfig.from_pretrained("xlm-roberta-base", num_labels=3)
model = AutoAdapterModel.from_pretrained("xlm-roberta-base", config=config)


#Here I am adding adapter
model.add_adapter("Telugu_code_mix_sentimentanalysis", config="pfeiffer")

model.add_classification_head("Telugu_code_mix_sentimentanalysis", num_labels=3)

#Here I am activating adapter
model.train_adapter("Telugu_code_mix_sentimentanalysis")

Some weights of XLMRobertaAdapterModel were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
#3rd training
training_arguments = TrainingArguments(
    learning_rate=best_run.hyperparameters['body_learning_rate'],
    num_train_epochs=best_run.hyperparameters['num_epochs'],
    per_device_train_batch_size=best_run.hyperparameters['batch_size'],
    per_device_eval_batch_size=best_run.hyperparameters['batch_size'],
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True,
    remove_unused_columns=False,
)

trainer = AdapterTrainer(
    model=model.to('cuda'),
    args=training_arguments,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss
200,0.969500
400,0.719800
600,0.557600
800,0.529100
1000,0.483100
1200,0.469600
1400,0.456100
1600,0.425400
1800,0.425900
2000,0.392700


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


TrainOutput(global_step=2954, training_loss=0.4855990877971507, metrics={'train_runtime': 231.8846, 'train_samples_per_second': 407.349, 'train_steps_per_second': 12.739, 'total_flos': 3988824519476160.0, 'train_loss': 0.4855990877971507, 'epoch': 7.0})

In [40]:
trainer.evaluate()

{'eval_loss': 0.46134185791015625,
 'eval_Accuracy': 0.8180201541197392,
 'eval_F1': 0.7927639663042679,
 'eval_Precision': 0.7912997594598257,
 'eval_Recall': 0.7946991585087136,
 'eval_runtime': 3.8995,
 'eval_samples_per_second': 865.245,
 'eval_steps_per_second': 27.183,
 'epoch': 7.0}

In [41]:
test_results = trainer.evaluate(test_custom_dataset)

In [42]:
print(test_results)

{'eval_loss': 0.5599409341812134, 'eval_Accuracy': 0.785, 'eval_F1': 0.7830406168387932, 'eval_Precision': 0.7853112323373431, 'eval_Recall': 0.785, 'eval_runtime': 3.5452, 'eval_samples_per_second': 846.207, 'eval_steps_per_second': 26.514, 'epoch': 7.0}


In [43]:
write_json('Telugu_code_mix_sentimentanalysis_hyper_parameter_3.json',best_run.hyperparameters)
files.download('Telugu_code_mix_sentimentanalysis_hyper_parameter_3.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [44]:
write_json('Telugu_code_mix_sentimentanalysis_result3.json',test_results)
files.download('Telugu_code_mix_sentimentanalysis_result3.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [45]:
model.save_adapter("./final_adapter", "Telugu_code_mix_sentimentanalysis")
!ls -lh final_adapter

total 5.8M
-rw-r--r-- 1 root root 1.1K Mar 31 14:58 adapter_config.json
-rw-r--r-- 1 root root  491 Mar 31 14:58 head_config.json
-rw-r--r-- 1 root root 3.5M Mar 31 14:58 pytorch_adapter.bin
-rw-r--r-- 1 root root 2.3M Mar 31 14:58 pytorch_model_head.bin


In [46]:
model.push_adapter_to_hub(
    repo_id="my-Telugu-codemix-xlmr-adapter3",
    adapter_name="Telugu_code_mix_sentimentanalysis",
)

pytorch_adapter.bin:   0%|          | 0.00/3.60M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model_head.bin:   0%|          | 0.00/2.37M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Blue7Bird/my-Telugu-codemix-xlmr-adapter3/commit/0d62308d337aa07abdc284db1e572ac894279105', commit_message='Upload model', commit_description='', oid='0d62308d337aa07abdc284db1e572ac894279105', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Blue7Bird/my-Telugu-codemix-xlmr-adapter3', endpoint='https://huggingface.co', repo_type='model', repo_id='Blue7Bird/my-Telugu-codemix-xlmr-adapter3'), pr_revision=None, pr_num=None)

In [48]:
#Here I am seeing the hyperparameters results
model1=load_json('/content/Telugu_code_mix_sentimentanalysis_hyper_parameter_1.json')
model2=load_json('/content/Telugu_code_mix_sentimentanalysis_hyper_parameter_2.json')
model3=load_json('/content/Telugu_code_mix_sentimentanalysis_hyper_parameter_3.json')
hyperparam_result = str(model1)+'\n'+str(model2)+'\n'+str(model3)
model1=load_json('/content/Telugu_code_mix_sentimentanalysis_result1.json')
model2=load_json('/content/Telugu_code_mix_sentimentanalysis_result2.json')
model3=load_json('/content/Telugu_code_mix_sentimentanalysis_result3.json')
performance_result = str(model1)+'\n'+str(model2)+'\n'+str(model3)
text = """*********hyperparameters********
{0}

***********performance*********
{1}
""".format(hyperparam_result,performance_result)
print(text)
with open('/content/Telugu_code_mix_sentiment_analysis_summary.txt',encoding='utf-8',mode='w') as f:
  f.write(text)
files.download('/content/Telugu_code_mix_sentiment_analysis_summary.txt')

*********hyperparameters********
{'body_learning_rate': 9.391380452265998e-05, 'num_epochs': 6, 'batch_size': 32, 'seed': 18, 'max_iter': 228, 'solver': 'lbfgs'}
{'body_learning_rate': 9.391380452265998e-05, 'num_epochs': 6, 'batch_size': 32, 'seed': 18, 'max_iter': 228, 'solver': 'lbfgs'}
{'body_learning_rate': 0.0004820940338739241, 'num_epochs': 7, 'batch_size': 32, 'seed': 24, 'max_iter': 122, 'solver': 'liblinear'}

***********performance*********
{'eval_loss': 0.5692409873008728, 'eval_Accuracy': 0.7663333333333333, 'eval_F1': 0.7640583350228094, 'eval_Precision': 0.7683793523227337, 'eval_Recall': 0.7663333333333333, 'eval_runtime': 4.1801, 'eval_samples_per_second': 717.683, 'eval_steps_per_second': 44.975, 'epoch': 8.0}
{'eval_loss': 0.5692409873008728, 'eval_Accuracy': 0.7663333333333333, 'eval_F1': 0.7640583350228094, 'eval_Precision': 0.7683793523227337, 'eval_Recall': 0.7663333333333333, 'eval_runtime': 4.1801, 'eval_samples_per_second': 717.683, 'eval_steps_per_second': 4

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Pfeiffer, J., Rücklé, A., Poth, C., Kamath, A., Vulić, I., Ruder, S., Cho, K., & Gurevych, I. (2020). AdapterHub: A Framework for Adapting Transformers. In Proceedings of the 2020 Conference on Empirical Methods in Natural Language Processing (EMNLP 2020): Systems Demonstrations (pp. 46--54). Association for Computational Linguistics. Online. Retrieved from https://www.aclweb.org/anthology/2020.emnlp-demos.7.

Poth, C., Sterz, H., Paul, I., Purkayastha, S., Englander, L., Imhof, T., Vulić, I., Ruder, S., Gurevych, I., & Pfeiffer, J. (2023). Adapters: A Unified Library for Parameter-Efficient and Modular Transfer Learning. In Proceedings of the 2023 Conference on Empirical Methods in Natural Language Processing: System Demonstrations (pp. 149--160). Association for Computational Linguistics. Singapore. Retrieved from https://aclanthology.org/2023.emnlp-demo.13.

https://huggingface.co/docs/setfit/en/how_to/hyperparameter_optimization